# Wash Trading

**HINT:** A wash trader could be someone sending and receiving the same NFT more than twice within the same day.

There's gotta be a better approach. perhaps seperating each transaction into TWO rows and transforming both from_address and to_address into the same field before performing a simple summation.

In [1]:
%load_ext dotenv
%dotenv
import os

import pandas as pd
from google.oauth2 import service_account

In [2]:
%%capture

credentials = service_account.Credentials.from_service_account_file(
    './details.json',
)

QUERY = """
SELECT address, COUNT(address) AS qty_meebits_wash_traded
FROM (
    SELECT to_address AS address, meebits_received.token_id, (received + sent) AS transfers FROM (
        SELECT to_address, value AS token_id, COUNT(value) AS received
        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address = '0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7'
        AND DATE(block_timestamp) = "2022-01-11"
        GROUP BY to_address, value
    ) meebits_received
    FULL OUTER JOIN (
        SELECT from_address, value AS token_id, COUNT(value) AS sent
        FROM `bigquery-public-data.crypto_ethereum.token_transfers`
        WHERE token_address = '0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7'
        AND DATE(block_timestamp) = "2022-01-11"
        GROUP BY from_address, value
    ) meebits_sent
    ON meebits_received.to_address = meebits_sent.from_address
    WHERE meebits_received.token_id = meebits_sent.token_id
    AND received + sent > 2
)
GROUP BY address
ORDER BY qty_meebits_wash_traded DESC
"""

data = pd.read_gbq(QUERY, project_id=os.getenv("PROJECT_ID"), credentials=credentials)

In [3]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///meebit_transfers.db', echo=False)
sqlite_connection = engine.connect()

sqlite_table = "Meebit_Transfers"
data.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

sqlite_connection.close()

In [4]:
data

,address,qty_meebits_wash_traded
0,0x0eba5e314b660503b13e47b733300d19d4648f06,3
1,0xebc23f01bb150efced718fe084238beb3777a716,3
2,0x565c4b34aac8448a7e1af9fd105461c46317f449,3
3,0x4789f41a2174eddb4a1371458e9e0cee48124562,2
4,0x534aa5f30805fc864360de06702d439c27fdea41,2
...,...,...
97,0x25de83caa6a2c4a07f7b4718faf5aa06d880c0cf,1
98,0x91ed35bbfad9ca76aa5573b06687a3214cb7707d,1
99,0x70be9cb0aeda8634f038201c00b37b2acb87a94d,1
100,0xf038946cef8566921e6631c9284d3fcd50309cc5,1


In [5]:
import sqlite3

In [6]:
QUERY = """
SELECT *
FROM meebit_transfers
"""

con = sqlite3.connect("meebit_transfers.db")
df = pd.read_sql_query(QUERY, con)
con.close()

df

,index,address,qty_meebits_wash_traded
0,0,0x0eba5e314b660503b13e47b733300d19d4648f06,3
1,1,0xebc23f01bb150efced718fe084238beb3777a716,3
2,2,0x565c4b34aac8448a7e1af9fd105461c46317f449,3
3,3,0x4789f41a2174eddb4a1371458e9e0cee48124562,2
4,4,0x534aa5f30805fc864360de06702d439c27fdea41,2
...,...,...,...
97,97,0x25de83caa6a2c4a07f7b4718faf5aa06d880c0cf,1
98,98,0x91ed35bbfad9ca76aa5573b06687a3214cb7707d,1
99,99,0x70be9cb0aeda8634f038201c00b37b2acb87a94d,1
100,100,0xf038946cef8566921e6631c9284d3fcd50309cc5,1
